# Обобщение текста

In [3]:
pip install torch transformers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.2 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 440 kB 44.6 MB/s eta 0:00:01
     |████████████████████████████████| 268 kB 66.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 44.7 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 48.3 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


Библиотека transformers предоставляет большое количество моделей для работы с разными типами данных для огромного спектра задач: 
1. Обобщение текста
2. Сегментация
3. Классификация
4. Перевод текста
5. Обобщение
6. Генерация

Сегодня нас интересуют модели для работы с текстами. 

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "IlyaGusev/rugpt3medium_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cpu")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

Эта модель работает следующим образом: получает на вход какой-то текст и выводит общую идею этой статьи.

In [17]:
article_text = "Актуальность проблемы. Электронная информация играет все большую роль во всех сферах жизни современного общества. В последние годы объем научно-технической текстовой информации в электронном виде возрос настолько, что возникает угроза обесценивания этой информации в связи с трудностями поиска необходимых сведений среди множества доступных текстов. Развитие информационных ресурсов Интернет многократно усугубило проблему информационной перегрузки. В этой ситуации особенно актуальными становятся методы автоматизации реферирования текстовой информации, то есть методы получения сжатого представления текстовых документов–рефератов (аннотаций). Постановка проблемы автоматического реферирования текста и, соответственно, попытки ее решения с использованием различных подходов предпринимались многими исследователями."
print(article_text)

Актуальность проблемы. Электронная информация играет все большую роль во всех сферах жизни современного общества. В последние годы объем научно-технической текстовой информации в электронном виде возрос настолько, что возникает угроза обесценивания этой информации в связи с трудностями поиска необходимых сведений среди множества доступных текстов. Развитие информационных ресурсов Интернета многократно усугубило проблему информационной перегрузки. В этой ситуации особенно актуальными становятся методы автоматизации реферирования текстовой информации, то есть методы получения сжатого представления текстовых документов–рефератов (аннотаций). Постановка проблемы автоматического реферирования текста и, соответственно, попытки ее решения с использованием различных подходов предпринимались многими исследователями.


Посмотрим, в чем, по мнению нашей модели, суть этого текста.

In [19]:
text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False, 
    padding=False,
    truncation=True
)["input_ids"]
input_ids = text_tokens + [tokenizer.sep_token_id]
input_ids = torch.LongTensor([input_ids]).to("cpu")

In [38]:
output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)

summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
summary = summary.split(tokenizer.sep_token)[1]
summary = summary.split(tokenizer.eos_token)[0]
print(f"Обобщение : {summary}")

Обобщение : Муравей и Стрекоза. Муравей поет, Стрекоза пляшет.


In [28]:
article_text = "Попрыгунья Стрекоза \
Лето красное пропела; \
Оглянуться не успела, \
Как зима катит в глаза. \
Помертвело чисто поле; \
Нет уж дней тех светлых боле, \
Как под каждым ей листком \
Был готов и стол, и дом. \
Все прошло: с зимой холодной \
Нужда, голод настает; \
Стрекоза уж не поет: \
И кому же в ум придет \
На желудок петь голодный! \
Злой тоской удручена, \
К Муравью ползет она: \
«Не оставь меня, кум милый! \
Дай ты мне собраться с силой \
И до вешних только дней \
Прокорми и обогрей!» — \
«Кумушка, мне странно это: \
Да работала ль ты в лето?» — \
Говорит ей Муравей. \
«До того ль, голубчик, было? \
В мягких муравах у нас \
Песни, резвость всякий час, \
Так, что голову вскружило». — \
«А, так ты...» — «Я без души \
Лето целое все пела». — \
«Ты все пела? Это дело: \
Так поди же, попляши!»"


article_text

'Попрыгунья Стрекоза Лето красное пропела; Оглянуться не успела, Как зима катит в глаза. Помертвело чисто поле; Нет уж дней тех светлых боле, Как под каждым ей листком Был готов и стол, и дом. Все прошло: с зимой холодной Нужда, голод настает; Стрекоза уж не поет: И кому же в ум придет На желудок петь голодный! Злой тоской удручена, К Муравью ползет она: «Не оставь меня, кум милый! Дай ты мне собраться с силой И до вешних только дней Прокорми и обогрей!» — «Кумушка, мне странно это: Да работала ль ты в лето?» — Говорит ей Муравей. «До того ль, голубчик, было? В мягких муравах у нас Песни, резвость всякий час, Так, что голову вскружило». — «А, так ты...» — «Я без души Лето целое все пела». — «Ты все пела? Это дело: Так поди же, попляши!»

In [29]:
text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False, 
    padding=False,
    truncation=True
)["input_ids"]
input_ids = text_tokens + [tokenizer.sep_token_id]
input_ids = torch.LongTensor([input_ids]).to("cpu")

In [30]:
output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)

summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
summary = summary.split(tokenizer.sep_token)[1]
summary = summary.split(tokenizer.eos_token)[0]
print(summary)

/opt/homebrew/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (800) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Муравей и Стрекоза. Муравей поет, Стрекоза пляшет.


# Задание 3

Попробуй применить нейронную сеть для других текстов. Посмотри, для каких текстов получилось хорошо, для каких не очень.